In [0]:
CREATE OR REPLACE TABLE jpm.chandra_silver.silver_customer AS
SELECT
    cust_id,
    name,
    CAST(age AS INT) AS age,
    gender,
    birthday,
    cast(zip as int) as zip,
    REPLACE(city, '_', ' ') AS city
FROM jpm.chandra_bronze.customer_bronze;


In [0]:
CREATE OR REPLACE TABLE jpm.chandra_silver.silver_transaction AS
SELECT DISTINCT
    cust_id,
    start_date,
    end_date,
    txn_id,
    date,
    year,
    month,
    day,
    expense_type,
    CAST(amt AS DECIMAL(10,2)) AS amt,
    REPLACE(city, '_', ' ') AS city
FROM jpm.naval_bronze.transaction_bronze;


In [0]:
%sql
create or replace  view jpm.chandra_silver.silver_customer_view as select * from jpm.chandra_silver.silver_customer;
create view jpm.chandra_silver.silver_transaction_view as select * from jpm.chandra_silver.silver_transaction;

In [0]:
%python
df_cust = spark.sql("select * from jpm.chandra_silver.silver_customer_view")
df_trans = spark.sql("select * from jpm.chandra_silver.silver_transaction_view")

df_joined = df_cust.join(df_trans, df_cust.cust_id == df_trans.cust_id)

display(df_joined)


In [0]:
CREATE OR REPLACE VIEW jpm.chandra_gold.df_joined AS
SELECT
    t.cust_id,
    c.name,
    t.start_date,
    t.end_date,
    t.txn_id,
    t.date,
    t.year,
    t.month,
    t.day,
    t.expense_type,
    t.amt,
    t.city,
    c.age,
    c.gender,
    c.birthday,
    c.zip
FROM
    jpm.chandra_silver.silver_transaction t
JOIN
    jpm.chandra_silver.silver_customer c
ON
    t.cust_id = c.cust_id;

In [0]:
CREATE OR REPLACE TABLE jpm.chandra_gold.top_customers AS
SELECT
    cust_id,
    name,
    SUM(amt) AS total_spent,
    RANK() OVER (ORDER BY SUM(amt) DESC) AS rank
FROM
    jpm.chandra_gold.df_joined
GROUP BY
    cust_id;

In [0]:
select * from jpm.chandra_gold.top_customers;